IMPORTING IMPORTANT LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy.linalg import multi_dot

MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


DATA PRE-PROCESSING FUNCTION

In [ ]:
def func(train_output,train_features,test_output,test_features):
  c0 = 0
  c1 = 0
  c2 = 0
  mean0 = 0
  mean1 = 0
  mean2 = 0
  for i in range(train_output.shape[0]):
      if train_output[i] == 1:
          c0 += 1
          mean0 += train_features[i]

      elif train_output[i] == 2:
          c1 += 1
          mean1 += train_features[i]
      else:
          c2 += 1
          mean2 += train_features[i]
  Py0 = c0 / train_output.shape[0]
  Py1 = c1 / train_output.shape[0]
  Py2 = c2 / train_output.shape[0]
  mean0 = mean0 / c0
  mean1 = mean1 / c1
  mean2 = mean2 / c2
  cov1 = np.cov((train_features - mean0).T)
  cov2 = np.cov((train_features - mean1).T)
  cov3 =  np.cov((train_features - mean2).T)
  y = []
  for i in range(test_features.shape[0]):
      p1 = np.exp(-0.5 * multi_dot([(test_features[i] - mean0).T,
                                    np.linalg.inv(cov1),
                                    (test_features[i] - mean0)]))
      cond_y1 = p1 / (np.power(2 * np.pi, 7 / 2) *
                      np.power(np.linalg.det(cov1), 0.5))
      
      p2 = np.exp(-0.5 * multi_dot([(test_features[i] - mean1).T,
                                    np.linalg.inv(cov2),
                                    (test_features[i] - mean1)]))
      
      cond_y2 = p2 / (np.power(2 * np.pi, 7 / 2) *
                      np.power(np.linalg.det(cov2), 0.5))
      p3 = np.exp(-0.5 * multi_dot([(test_features[i] - mean2).T,
                                    np.linalg.inv(cov3),
                                    (test_features[i] - mean2)]))
      
      cond_y3 = p3 / (np.power(2 * np.pi, 7 / 2) *
                      np.power(np.linalg.det(cov3), 0.5))
      
      P1 = cond_y1 * Py0 / (cond_y1 * Py0 + cond_y2 * Py1 + cond_y3 * Py2)
      P2 = cond_y2 * Py1 / (cond_y1 * Py0 + cond_y2 * Py1 + cond_y3 * Py2)
      P3 = cond_y3 * Py2 / (cond_y1 * Py0 + cond_y2 * Py1 + cond_y3 * Py2)
      m = max(P1, P2, P3)
      if m == P1:
          y.append(1)
      elif m == P2:
          y.append(2)
      else:
          y.append(3)
  cnt=0
  for i in range(test_features.shape[0]):
    if y[i]==test_output[i]:
      cnt+=1
  return print("Individual Accuracy :",round(cnt*100/test_features.shape[0],4),'%')

MAXIMUM A POSTERIORI ALGORITHM

In [ ]:
from random import Random
dataset=pd.read_csv("/content/gdrive/MyDrive/NNFL_ASSIGNMENT_1/data_q6_q7.txt",names=["a", "b", "c", "d", "e", "f", "g", "output"],delimiter="\t",error_bad_lines=False)
Random(50).shuffle(dataset.values)
dataset.reindex(np.random.permutation(dataset.index))
data1 = dataset.dropna()
data = data1.reset_index()

X = data.drop('output',axis=1)
y = data['output']

def five_fold_batch_L2(X,y):
  
  m = len(y)
  for i in range(5):
    X_test = X[int(i*m*0.2):int((i+1)*m*0.2)]
    X_train = pd.concat([X, X_test, X_test]).drop_duplicates(keep=False)
    y_test = y[int(i*m*0.2):int((i+1)*m*0.2)]
    y_train = y.drop(y_test.index)

    train_features = np.asarray(X_train)
    test_features = np.asarray(X_test)
    train_output = np.asarray(y_train)
    test_output = np.asarray(y_test)

    x = []
    mean = []
    std = []
    for i in range(train_features.shape[1]):
        x.append((train_features[:, i] - np.mean(train_features[:, i])) /
                np.std(train_features[:, i]))
        mean.append(np.mean(train_features[:, i]))
        std.append(np.std(train_features[:, i]))

    train_features = np.array(x).T

    x = []
    for i in range(test_features.shape[1]):
        x.append((test_features[:, i] - mean[i]) / std[i])

    test_features = np.array(x).T

    func(train_output,train_features,test_output,test_features)  

  return

ACCURACY

In [ ]:
five_fold_batch_L2(X,y)

Individual Accuracy : 100.0 %
Individual Accuracy : 100.0 %
Individual Accuracy : 97.5 %
Individual Accuracy : 77.5 %
Individual Accuracy : 92.5 %


OVERALL ACCURACY : 93.5%